In [25]:
# step 1: import dem libraries kiddos!
import json
import pandas as pd
import numpy as np
import nltk as nltk
import spacy
import os
import re 
import sklearn 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from collections import Counter
import nltk
#nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from nltk.stem.porter import *
porter_stemmer = PorterStemmer()
import nltk.data
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
from nltk.tokenize import TreebankWordTokenizer
treebank_tokenizer = TreebankWordTokenizer()


In [26]:
# import the diaglogue Json Files!!!!!

wd = os.getcwd() 
# grab the locations of those lil jsons
json_files = [pos_json for pos_json in os.listdir(wd + '\\diag_jsons\\') if pos_json.endswith('.json')]

# read those guys in!

# the first step will be to READ each file, then the NEXT step will be to store it in something that will EVENTUALLY 
# be converted into a dataframe (actually, series)

# so I reckon the way to start this bad boy off is to just store them all in a SHINY NEW DICTIONARY

# and let the key be the movie name  


In [24]:
# I commented out an example. 
#with open(wd+ '\\diag_jsons\\'+json_files[0], "r") as read_file:
#    data = json.load(read_file)
#json_files[0].replace('.json','')

In [27]:
# found this on stack overflow
def remove_multiple_strings(cur_string, replace_list,low= False):
    for cur_word in replace_list:
        if low == False:
            cur_string = cur_string.replace(cur_word, '')
        else:
            cur_string = cur_string.replace(cur_word.lower(), '')
    return cur_string

In [28]:
# create a new dictionary where each key will be the movie and each value will contain all of the 
mov_dict = {}
char_series = pd.Series()
for fil in json_files:
    # read the json file from the directory 
    with open(wd+ '\\diag_jsons\\'+fil, "r") as read_file:
        data = json.load(read_file)
        
    mov_dict[fil.replace('.json','')] = data
    # now that we have done this 
    # we may as well start loading up our pandas series with the character texts.
    # so we will start iterating through each character in the new json file 
    # added to our dictionary and make each character a new iterm in our series 
    # which will be indexed by the title of the script/movie and the character's name 
    # with four '_'s between them 
    # so we can do easy retrieval of which character was from which movie :)
    mov_chars = mov_dict[fil.replace('.json','')]['dialogues'].keys()
    for char in mov_dict[fil.replace('.json','')]['dialogues'].keys():
        #char_series.at[char+'____'+fil.replace('.json','')] = mov_dict[fil.replace('.json','')]['dialogues'][char]
        char_series.at[char+'____'+fil.replace('.json','')] = re.sub("\\n\[\d+\]",'',mov_dict[fil.replace('.json','')]['dialogues'][char])
        # until I think of a better way of replacing the names, 
        # I have to make the entire text lowercase 
        # so that way I get all instances of the character names being used.... 
        char_series.at[char+'____'+fil.replace('.json','')] = char_series.at[char+'____'+fil.replace('.json','')].lower()
        # now we want to remove the names of the characters, because those seemed to be too important in creating clusters
        char_series.at[char+'____'+fil.replace('.json','')] = remove_multiple_strings(char_series.at[char+'____'+fil.replace('.json','')],mov_chars,low=True)
        
        
        tok_chars = sentence_tokenizer.tokenize(char_series.at[char+'____'+fil.replace('.json','')])
            # so... we need to do this for every sentence.... YAY.... 
        lemz = ''
        # I think I need to get rid of punctuation unless lemmatization gets rid of it 
        for sentence in tok_chars:
            toks = treebank_tokenizer.tokenize(sentence)
            lemmatized_words = [wordnet_lemmatizer.lemmatize(token) for token in toks]
    
            lemz += " ".join(lemmatized_words) + ' '
        char_series.at[char+'____'+fil.replace('.json','')] = lemz
        # we get rid of those pesky \n[number] patterns that denote a line change 
        # which I suppose I can add back IN as a second column if necessary , but that's just going to be unnecessary for the clustering!

# I'm guessing that every thime there is a new line it will go with that kind of scheme 
# so I suppose my job will be to remove those bois if they still occur after tokenization.... hoping they just dissapear, lmao !!!

# also, very nice, I can iterate over the ['dialogues.keys()']
# this is VERY fun!!!! 


In [29]:
char_series.head()

KAT____10-Things-I-Hate-About-You_script        leave it i said , leave it ! why did n't we ju...
PATRICK____10-Things-I-Hate-About-You_script    i missed you . it wa a bratwurst . i wa eating...
BIANCA____10-Things-I-Hate-About-You_script     did you change your hair ? you might wan na th...
CAMERON____10-Things-I-Hate-About-You_script    i do n't think so , ma'am so they tell me ... ...
MICHAEL____10-Things-I-Hate-About-You_script    you the new guy ? c'mon . i 'm supposed to giv...
dtype: object

In [37]:
#miniboi = pd.Series()
#for i in mov_dict['10-Things-I-Hate-About-You_script']['dialogues'].keys():
#    miniboi.at[i+'____''10-Things-I-Hate-About-You_script'] = mov_dict['10-Things-I-Hate-About-You_script']['dialogues'][i]
#miniboi



# ok, so I suppose we will need to figure out how we will create the new index for the Pandas.Series
# heck, I suppose I could even add a movie title AFTER... but meh, I'm lazy... I think I'll just leave it as the series for now! 


In [126]:
#thingz = sentence_tokenizer.tokenize(miniboi[0])
# so... we need to do this for every sentence.... YAY.... 
#lemz = ''
# I think I need to get rid of punctuation unless lemmatization gets rid of it 
#for sentence in thingz:
#    toks = treebank_tokenizer.tokenize(sentence)
#    lemmatized_words = [wordnet_lemmatizer.lemmatize(token) for token in toks]
    
#    lemz += " ".join(lemmatized_words) + ' '


In [30]:
#lemz

In [31]:
# for examples sake... run from the line that says mniboi  until the one that says miniboi[0]
#miniboi

#miniboi[0] = re.sub("\\n\[\d+\]",'',miniboi[0])
#         char_series.at[char+'____'+fil.replace('.json','')] = re.sub("\\n\[\d+\]",'',mov_dict[fil.replace('.json','')]['dialogues'][char])

#miniboi[0]

# you want to lemmatize the the text or tokenize etc, 

# tokenize by sentence 1st...then apply the lemmatization to each sentence

# then we have to find a way to concatenate or .join everything together across all of the lists!!! 
#miniboi[0]
# Do some Data preprocessing on the text
# maybe tokenizing, stemming or lemmatizing will get rid of the bad woyds!
# so now I know to do this to the text to preprocess it, again, I can do it in the initial loop, or after. 
# it's kinda whatever. 

# time to lemmatize!!!!
# or friggin tokenize, at least!
#thingz = sentence_tokenizer.tokenize(miniboi[0])
#print(thingz)
#lemz = []
#for sentence in thingz:
#    lemmatized_words = [wordnet_lemmatizer.lemmatize(word) for word in sentence]
#    lemz += lematized_words
#lemmatized_words
#" ".join(["John", "Charles", "Smith"])

# Time to get those term counts!!!!!
#lemz
#miniboi[0]

In [32]:
# LSA, LSA!!!!
# wondeirng what the count_vecotrizer does??? Here's the basic documentation: 
# Convert a collection of text documents to a matrix of token counts
#This implementation produces a sparse representation of the counts using scipy.sparse.csr_matrix.
#If you do not provide an a-priori dictionary and you do not use an analyzer that does some kind of feature selection then the number of features will be equal to the vocabulary size found by analyzing the data.
count_vectorizer = CountVectorizer(stop_words='english', max_features=10000)
text_sample = char_series.as_matrix()
# so we essenatially create our document_term matrix.... 
#display(small_text_sample)
document_term_matrix = count_vectorizer.fit_transform(text_sample)
#display(document_term_matrix)

C:\Users\seanp\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys


In [33]:
# Let's reduce the dimensionality of that Document term matrix and do some clustering! 

# I"m doing this for 2 reasons: 1, I essentially copy pasted from Stephen 

#2:.... we have a lot of data.... let's make the run time as fast as possible!


In [34]:
# Define helper functions, These are from Stephen
def get_keys(topic_matrix):
    '''returns an integer list of predicted topic categories for a given topic matrix'''
    keys = []
    for i in range(topic_matrix.shape[0]):
        keys.append(topic_matrix[i].argmax())
    return keys
###################################################################################
def keys_to_counts(keys):
    '''returns a tuple of topic categories and their accompanying magnitudes for a given list of keys'''
    count_pairs = Counter(keys).items()
    categories = [pair[0] for pair in count_pairs]
    counts = [pair[1] for pair in count_pairs]
    return (categories, counts)

##############################################################################
def get_top_n_words(n, keys, document_term_matrix, count_vectorizer):
    '''returns a list of n_topic strings, where each string contains the n most common 
        words in a predicted category, in order'''
    top_word_indices = []
    for topic in range(n_topics):
        temp_vector_sum = 0
        for i in range(len(keys)):
            if keys[i] == topic:
                temp_vector_sum += document_term_matrix[i]
        temp_vector_sum = temp_vector_sum.toarray()
        top_n_word_indices = np.flip(np.argsort(temp_vector_sum)[0][-n:],0)
        top_word_indices.append(top_n_word_indices)   
    top_words = []
    for topic in top_word_indices:
        topic_words = []
        for index in topic:
            temp_word_vector = np.zeros((1,document_term_matrix.shape[1]))
            temp_word_vector[:,index] = 1
            the_word = count_vectorizer.inverse_transform(temp_word_vector)[0][0]
            topic_words.append(the_word.encode('ascii').decode('utf-8'))
        top_words.append(" ".join(topic_words))         
    return top_words

In [35]:
count_vectorizer = CountVectorizer(stop_words='english', max_features=10000)
text_sample = char_series.as_matrix()
# so we essenatially create our document_term matrix.... 
#display(small_text_sample)
document_term_matrix = count_vectorizer.fit_transform(text_sample)
display(document_term_matrix)
for i in range(5,31):
    n_topics = i

    lsa_model = TruncatedSVD(n_components=n_topics)
    lsa_topic_matrix = lsa_model.fit_transform(document_term_matrix)
   # display(lsa_topic_matrix)

    lsa_keys = get_keys(lsa_topic_matrix)
    lsa_categories, lsa_counts = keys_to_counts(lsa_keys)

    top_n_words_lsa = get_top_n_words(20, lsa_keys, document_term_matrix, count_vectorizer)

    for i in range(len(top_n_words_lsa)):
        print("Topic {}: ".format(i), top_n_words_lsa[i])
    print('********')

C:\Users\seanp\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


<11493x10000 sparse matrix of type '<class 'numpy.int64'>'
	with 1379968 stored elements in Compressed Sparse Row format>

Topic 0:  know wa just like ll did got want right look think ve going come good time oh say let yeah
Topic 1:  wa did time year man day life like sir yes mr know ha said mother just god night father good
Topic 2:  look ou ll nd ha door ve hand turn er eye come open face revision head men pull 07 room
Topic 3:  oh yes hi okay yeah god sorry dad hey la absolutely hello thank going shut good vous think texas really
Topic 4:  th just hr sh thy know tht ll gt look wht lik nd got hav thn fo thr wa hv
********
Topic 0:  know wa just like ll did got want right look think ve going come good time oh say let yeah
Topic 1:  wa did time man year day life like yes ha know sir said mr mother just night god father came
Topic 2:  look ha door hand turn eye nd come open ll face revision head 07 away hm room pull car wth
Topic 3:  oh yes hi okay yeah sorry god dad hey la shut thank good going vous hello really texas en think
Topic 4:  th just hr sh thy know tht ll gt wht look lik nd got hav thn fo thr wa

Topic 3:  oh okay hi dad sorry yes going yeah god hey really texas shake meeting mum just think la thank thanks
Topic 4:  th just hr sh thy know tht ll gt look wht lik got nd hav thn fo thr hv wa
Topic 5:  ll nd ve ou er bu ere mr ur ell le ju men ne ee nk cn mn dd don
Topic 6:  oh sir mr yes ha come yeah know uh hey na ou god ll wan oz great gon good wa
Topic 7:  schultz dr yes man like horse say good know big just little nigger mr slave ai white got black em
Topic 8:  know toy penny whistle want right doe ca going did fuck line guy frank father cliff really taking goddamn work
Topic 9:  07 revision pink 20 white 19 10 29 yellow 25 11 23 40 45 62 21 49 16 27 30
Topic 10:  ou yu ha nd er ll ee ve ju ih hi fo hav man ro oo od hey ke fuck
Topic 11:  nd red dr rum ou er schultz look ey ee jones bu ur ne rg ll rm ve door mn
Topic 12:  yes did fuck fuckin oh acknowledged na request little gon dr schultz ll say fucking shit said look doing tell
Topic 13:  got right oh yeah going big ya hey o

Topic 6:  ha oh mr sir fo yes ll hav ou come know don hey uh hy ill old hn yeah na
Topic 7:  schultz dr like man yes horse say good know big little just nigger slave ai white black got em look
Topic 8:  know penny toy whistle want ca doe right did going frank fuck line father guy goddamn year week come cliff
Topic 9:  07 revision pink 20 white 19 10 29 yellow 25 49 40 62 11 45 23 21 59 46 22
Topic 10:  ou ha yu er ll man ee ve ro ju fuck od og oo nd ih hav oud ke dd
Topic 11:  nd yu red dr hi rum bu ne ee er hey en dn ou ju ih ve ey hee kn
Topic 12:  did fuck yes fuckin oh na gon little ll fucking dr schultz say shit said right doing mr bos tell
Topic 13:  got right oh yeah going big ya hey okay ve ta huh lot bird cuz cow mr coffee seven way
Topic 14:  sir yes mr ha just captain ve order thank need minute president say let ship percent course second starfleet report
Topic 15:  mommy kill help momma just man come martin confirmed leave save pack billy got oh stop miss water cold breathe

Topic 4:  th just hr sh thy know tht ll gt look wht lik got nd hav thn fo thr hv wa
Topic 5:  ll nd ve ou er bu ere ur mr ell ju le men ne nk mn ee cn dd don
Topic 6:  oh ha mr sir ll fo hav yes come hy uh old ill don yeah hn man know god say
Topic 7:  schultz dr like man horse yes know say big little nigger just slave good white ai got black em dollar
Topic 8:  know whistle toy penny want ca doe right did going guy father line come fuck frank man wo said need
Topic 9:  07 revision pink 20 white 19 10 29 yellow 25 62 49 21 40 11 45 16 22 23 30
Topic 10:  ha fuck man ll majsy fo qun hav fucking kno ih ho shit don ill old nd mo lady know
Topic 11:  nd er dr yu bu ee schultz ey look ne ur ll rm en dn mn jones ke mne le
Topic 12:  fuck did fuckin na oh gon little ll fucking schultz dr shit say doing yes right mr cop tell bitch
Topic 13:  got right oh going yeah hey big ve ta ya guy look lot huh kid tried stuff okay shit cow
Topic 14:  sir ha mr yes captain just ve order president ship minu

Topic 13:  got right oh going yeah hey big ve ta look guy ya lot kid huh okay tried cow bird shit
Topic 14:  sir yes ha just captain signal ve ship minute thank starfleet second need contact attention report let engine mr order
Topic 15:  mommy kill oh help shit god martin got hear breathe ca water pack just confirmed come save say use lost
Topic 16:  want tat wat good ti come daddy av okay know ust look lik ya ll wa wit ow love say
Topic 17:  right just left destination come enter momma point ll change yeah man home roy mr musketeer haven kay sent hotel
Topic 18:  okay kill going ll hide hey dad nvr ow av wake elf know good state hop hit pretty help let
Topic 19:  dragon yes toothless final sir 13 draft sanders 02 deblois 2010 like red head just eye look turn know okay
Topic 20:  kill beast loser want dad caveman let nigger present yeah happen state test oh death excused burn grand finish hero
Topic 21:  ha yu wha hey wa hee wh ge know ke ou ee ae ve ju don hnk ay guy aou
Topic 22:  m

Topic 13:  got right oh going yeah big hey ta huh ve ya lot guy tried cow look shit cuz bird stuff
Topic 14:  sir ha yes ve ship just need minute attention contact engine second order let percent atmosphere report general mr hull
Topic 15:  mommy ca kill oh help shit god got hear martin come water pack save just start say breathe bring boy
Topic 16:  want tat wat good come ti okay know daddy av look ust lik ya ll wa wit say ar stay
Topic 17:  just right destination come change point enter momma man yeah ll haven roy sent musketeer kay home boy 16 mate
Topic 18:  okay going hide hey dad hop wake elf ll pretty state hit man guardian tomorrow 12 avid let mistress father
Topic 19:  sir yes dragon toothless like draft 13 deblois 2010 02 sanders final just good know head look okay come did
Topic 20:  red rum dragon kill toothless final beast like sanders 02 deblois 13 draft 2010 loser want head eye just cloud
Topic 21:  ha wha hey wa yu know wh ou hee ae ge ke ve don ay hnk guy jus wan na
To

In [36]:
# Now we will do LSA with tokenization instead of lemmatization, just so that the outputted words make more sense!
# Hopefully there's a way to do this with sequences of 2-3 words, because some of these are gibberish.


In [22]:
# create a new dictionary where each key will be the movie and each value will contain all of the 
mov_dict = {}
char_series = pd.Series()
for fil in json_files:
    # read the json file from the directory 
    with open(wd+ '\\diag_jsons\\'+fil, "r") as read_file:
        data = json.load(read_file)
        
    mov_dict[fil.replace('.json','')] = data
    # now that we have done this 
    # we may as well start loading up our pandas series with the character texts.
    # so we will start iterating through each character in the new json file 
    # added to our dictionary and make each character a new iterm in our series 
    # which will be indexed by the title of the script/movie and the character's name 
    # with four '_'s between them 
    # so we can do easy retrieval of which character was from which movie :)
    mov_chars = mov_dict[fil.replace('.json','')]['dialogues'].keys()
    for char in mov_dict[fil.replace('.json','')]['dialogues'].keys():
        #char_series.at[char+'____'+fil.replace('.json','')] = mov_dict[fil.replace('.json','')]['dialogues'][char]
        char_series.at[char+'____'+fil.replace('.json','')] = re.sub("\\n\[\d+\]",'',mov_dict[fil.replace('.json','')]['dialogues'][char])
        # until I think of a better way of replacing the names, 
        # I have to make the entire text lowercase 
        # so that way I get all instances of the character names being used.... 
        char_series.at[char+'____'+fil.replace('.json','')] = char_series.at[char+'____'+fil.replace('.json','')].lower()
        # now we want to remove the names of the characters, because those seemed to be too important in creating clusters
        char_series.at[char+'____'+fil.replace('.json','')] = remove_multiple_strings(char_series.at[char+'____'+fil.replace('.json','')],mov_chars,low=True)
        
        
        tok_chars = sentence_tokenizer.tokenize(char_series.at[char+'____'+fil.replace('.json','')])
            # so... we need to do this for every sentence.... YAY.... 
        lemz = ''
        # I think I need to get rid of punctuation unless lemmatization gets rid of it 
        for sentence in tok_chars:
            toks = treebank_tokenizer.tokenize(sentence)
            #lemmatized_words = [wordnet_lemmatizer.lemmatize(token) for token in toks]
    
            lemz += " ".join(toks) + ' '
        char_series.at[char+'____'+fil.replace('.json','')] = lemz
        # we get rid of those pesky \n[number] patterns that denote a line change 
        # which I suppose I can add back IN as a second column if necessary , but that's just going to be unnecessary for the clustering!

# I'm guessing that every thime there is a new line it will go with that kind of scheme 
# so I suppose my job will be to remove those bois if they still occur after tokenization.... hoping they just dissapear, lmao !!!

# also, very nice, I can iterate over the ['dialogues.keys()']
# this is VERY fun!!!! 


In [23]:
count_vectorizer = CountVectorizer(stop_words='english', max_features=10000)
text_sample = char_series.as_matrix()
# so we essenatially create our document_term matrix.... 
#display(small_text_sample)
document_term_matrix = count_vectorizer.fit_transform(text_sample)
display(document_term_matrix)

n_topics = i

lsa_model = TruncatedSVD(n_components=n_topics)
lsa_topic_matrix = lsa_model.fit_transform(document_term_matrix)
   # display(lsa_topic_matrix)

lsa_keys = get_keys(lsa_topic_matrix)
lsa_categories, lsa_counts = keys_to_counts(lsa_keys)

top_n_words_lsa = get_top_n_words(20, lsa_keys, document_term_matrix, count_vectorizer)

for i in range(len(top_n_words_lsa)):
    print("Topic {}: ".format(i), top_n_words_lsa[i])


C:\Users\seanp\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


<11493x10000 sparse matrix of type '<class 'numpy.int64'>'
	with 1389213 stored elements in Compressed Sparse Row format>

Topic 0:  know just like ll did got want right ve think going good oh come time let look yeah ca na
Topic 1:  na gon got yeah man fuck wan shit oh hey come ai ya fuckin ta ass right know money okay
Topic 2:  oh yeah hi uh huh yes god sorry okay shut really cool hey nice meeting great just texas bunny dad
Topic 3:  th just hr sh thy tht ll gt know wht lik hav thn thr nd fo hv ws got oh
Topic 4:  looks door turns eyes hand room away face car head takes just look pulls sees starts open looking comes grabs
Topic 5:  ll nd hs ve ou ha er hm bu lke wha dd know don wth ur ere ell cn gong
Topic 6:  yes oh sir dr mr man master know come black bird enterprise schultz little house say authorization daddy boy good
Topic 7:  know robert contract hs want lady say did cliff attorney draw money looking happens ca think took tell right grandma
Topic 8:  07 revision pink 20 white 19 10 29 yellow 40 11 25 45 22 21 16 30 26 23 33
Topic 9:  hs wth hm ths nto just know lke man got dd looks turns tme wll thn